# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# List of input files
file_path_list = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list.extend(glob.glob(os.path.join(root,'*')))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# total number of rows 
#print(len(full_data_rows_list))

# list of event data rows
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6914


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a session with class = SimpeStrategy and replication factor across cluster = 1
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_library
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set current session to music_library keyspace
try:
    session.set_keyspace('music_library')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Table to retrieve artist, song title and song's length that was played in a particular session at a particular position
query = """
CREATE TABLE song_info_filteredOn_session_itemInSession (
    artist text, item_in_session int, length float, session_id int, song text,
    PRIMARY KEY (session_id, item_in_session)
    );
"""
try:
    session.execute("DROP TABLE IF EXISTS song_info_filteredOn_session_itemInSession;")
    session.execute(query)
except Exception as e:
    print(e)                   

In [9]:
# Read the intermediate csv file and add data to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert statement to insert data into table
        query = "INSERT INTO song_info_filteredOn_session_itemInSession (artist, item_in_session, length, session_id, song)"
        query = query + "values (%s, %s, %s, %s, %s);"

        # Execute the insert statement
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM song_info_filteredOn_session_itemInSession WHERE session_id = 338 and item_in_session = 4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
# Table to retrieve artist, user information and song title played by a user in a particular session
query = """
CREATE TABLE user_session_song_info (
     artist text, first_name text, item_in_session int, last_name text,
     session_id int,song text,user_id int, PRIMARY KEY ((user_id, session_id), item_in_session)
     );
"""
try:
    session.execute("DROP TABLE IF EXISTS user_session_song_info;")
    session.execute(query)
except Exception as e:
    print(e)                   

In [12]:
# Read the intermediate csv file and add data to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert statement to insert data into table
        query = "INSERT INTO user_session_song_info (artist, first_name, item_in_session, last_name, session_id, song, user_id)"
        query = query + "values (%s, %s, %s, %s, %s, %s, %s);"
        
        # Execute the insert statement
        session.execute(query, (line[0], line[1], int(line[3]), line[4], int(line[8]), line[9], int(line[10])))

In [13]:
# SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, first_name, last_name FROM user_session_song_info WHERE user_id = 10 and session_id = 182;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


In [14]:
# Table to retrieve user information who listens to a particular song
query = """
CREATE TABLE user_info_filteredOn_song (
     first_name text, last_name text, song text, PRIMARY KEY (song));
"""
try:
    session.execute("DROP TABLE IF EXISTS user_info_filteredOn_song;")
    session.execute(query)
except Exception as e:
    print(e)                 

In [15]:
# Read the intermediate csv file and add data to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert statement to insert data into table
        query = "INSERT INTO user_info_filteredOn_song (first_name, last_name, song)"
        query = query + "values (%s, %s, %s);"

        # Execute the insert statement
        session.execute(query, (line[1], line[4], line[9]))

In [16]:
# SELECT statement to verify the data was entered into the table
query = "SELECT first_name, last_name FROM user_info_filteredOn_song WHERE song = 'All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [17]:
# drop all the tables
try:
    session.execute("DROP TABLE IF EXISTS song_info_filteredOn_session_itemInSession;")
    session.execute("DROP TABLE IF EXISTS user_session_song_info;")
    session.execute("DROP TABLE IF EXISTS user_info_filteredOn_song;")
except Exception as e:
    print(e)   

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()